In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

In [2]:
fb_annot_path = '../../clips_no_wiggle/fbk_cuts_no_wiggle_0_4210.csv'
transcriptions_dir = 'results/extract_dialogue/transcriptions/'
aligned_transcriptions_dir = 'results/extract_dialogue/aligned_transcriptions/'


In [3]:
def align_human_annotations(fb_annot, transcriptions):
    transcriptions = transcriptions.copy()
    transcriptions.reset_index(drop=True, inplace=True)
    transcriptions['human_annotations'] = None
    transcriptions['human_annotation_times'] = None
    transcriptions['trainee_responses'] = None
    
    fb_annot = fb_annot[fb_annot['Case'] == 1]
    fb_annot.reset_index(drop=True, inplace=True)
    for i in tqdm(range(len(transcriptions))):
        start_sec = transcriptions.loc[i, 'start']
        end_sec = transcriptions.loc[i, 'end']
        
        k = 0
        human_annotations = []
        human_annotation_times = []
        trainee_responses = []
        for j in range(len(fb_annot)):
            fbk_secs = sum([a*b for a,b in zip([3600, 60, 1], map(int, fb_annot.loc[j, 'fbk_time'].split(':')))])
            if fbk_secs >= start_sec and fbk_secs <= end_sec:
                time = fb_annot.loc[j, 'fbk_time']
                time_str = ':'.join([f"{int(x):02d}" for x in time.split(':')])
                
                human_annotations.append(fb_annot.loc[j, 'Dialogue'])
                human_annotation_times.append(time_str)
                trainee_responses.append({'r_t_verb': int(fb_annot.loc[j, 'r_t_verb']), 'r_t_beh': int(fb_annot.loc[j, 'r_t_beh']), 'r_t_clarify': int(fb_annot.loc[j, 'r_t_clarify'])})
                k += 1
        
        transcriptions.at[i, 'human_annotations'] = human_annotations
        transcriptions.at[i, 'human_annotation_times'] = human_annotation_times
        transcriptions.at[i, 'trainee_responses'] = trainee_responses
    
    return transcriptions

def run_and_save(case_id):
    os.makedirs(aligned_transcriptions_dir, exist_ok=True)
    
    transcriptions_path = os.path.join(transcriptions_dir, f'LFB{case_id}_full.csv')
    aligned_transcriptions_save_path = os.path.join(aligned_transcriptions_dir, f'LFB{case_id}_full.csv')
    
    fb_annot = pd.read_csv(fb_annot_path, index_col=0)
    transcriptions = pd.read_csv(transcriptions_path)

    aligned_transcriptions = align_human_annotations(fb_annot, transcriptions)
    aligned_transcriptions.to_csv(aligned_transcriptions_save_path)


In [4]:
for i in range(1, 35):
    run_and_save(i)

  0%|          | 0/1711 [00:00<?, ?it/s]

100%|██████████| 1952/1952 [00:02<00:00, 765.30it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'results/extract_dialogue/transcriptions/LFB34_full.csv'